# Caveats and specific cases

Here we describe some particular cases you might find when using the light curves in KBonus-BKG.

In [15]:
# basic imports
import numpy as np
import matplotlib.pyplot as plt

import lightkurve as lk
from astropy.table import Table
from astropy.io import fits

## 1. Sources with partial data

Some sources in KBonus-BKG were extracted with partial data, i.e. using pixels in the wing of the PSF, this is due to sources falling out of the edge of the TPF or because of saturated pixel rejection removed the central pixels of a bright source. 
The light curves provide the `psffrac` metric as the fraction of the PSF on the TPF used for photometry. This value is close to 1 for sources with full coverage and close to 0 for sources with partial data. LightKurve (v2.5.0) will show a warning when a source has `psffrac < 0.6` and also for sources brighter than $K_{p} < 12$ that produced saturated pixels that were avoided for model fitting.

In [2]:
lc = lk.search_lightcurve("Kepler-10", mission="Kepler", author="KBONUS-BKG").download()

/Users/jorgemarpa/Work/public_repos/lightkurve/src/lightkurve/search.py:480: LightkurveWarning: Warning: 19 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
Kepler magnitude is bright (less than 12), indicating the target is saturated. KBONUS-BKG data is invalid for saturated targets.
PSF Fraction is low, indicating not all of the flux is captured in the PSF model. This data may be unreliable.


In [3]:
lc.meta["KEPMAG"]

10.9610004425049

In [4]:
lcq = lk.read(lc.filename, quarter=2)
lcq

Kepler magnitude is bright (less than 12), indicating the target is saturated. KBONUS-BKG data is invalid for saturated targets.
PSF Fraction is low, indicating not all of the flux is captured in the PSF model. This data may be unreliable.


time,flux,flux_err,cadenceno,sap_flux,centroid_col,sap_flux_err,centroid_column,centroid_row,sap_quality,sap_bkg,red_chi2,psf_flux_nova,psf_flux_err_nova
,electron / s,electron / s,,electron / s,pix,electron / s,pix,pix,,,,electron / s,electron / s
Time,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
169.51974208513275,296901.4407917318,35.48463688277593,2965,55391.66015625,655.449604353118,7.411260604858398,655.449604353118,250.63795690575847,256,0.0,245983.1902685441,300947.70202336845,36.107478828668675
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258.4670207872987,302200.2129394214,37.869976908909734,7318,52152.3203125,655.4035519418761,7.2654314041137695,655.4035519418761,250.33718467545543,0,0.0,305202.6464676686,278168.5981922249,35.1327248641398


In [5]:
print(f"Quarter {lcq.quarter} \tPSFFRAC = {lcq.psffrac}")

Quarter 2 	PSFFRAC = 0.1763822145682909


Sources extracted with partial data, due to pixel saturation or off TPF, may result in unreliable photometry and light curves. PSF photometry could be noisier, while aperture photometry will be partial.

## 2. Empty PSF photometry

As described in [Martinez-Palomera et al. 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv231017733M/abstract), the PSF is fitted with a linear model and it does not have constraints in the predicted values rather than a Normal prior distribution around the Gaia DR3 flux value. This could result in predicted negative fluxes. To mitigate this issue the extraction pipeline iterated up to 3 times reducing the prior width to enforce positive flux values. But this does not guarantee good results and some sources can still get predicted negative fluxes. 
This happens typically for highly blended sources or targets near saturated pixels. 
In most cases these sources were removed from the extracted catalog, but some had successful aperture photometry estimation. We kept those.
This results in some sources having the PSF flux columns empty and the aperture flux complum populated.
Here's an example

In [68]:
hdul = fits.open("https://archive.stsci.edu/hlsps/kbonus-bkg/lcs/2051/2051766354744241536/hlsp_kbonus-bkg_kepler_kepler_gaia-dr3-2051766354744241536_kepler_v1.0_lc.fits")
hdul.info()

Filename: /Users/jorgemarpa/.astropy/cache/download/url/0159dff9ad88f67f55242999c03c282a/contents
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      48   ()      
  1  LIGHTCURVE_STITCHED    1 BinTableHDU     40   65277R x 11C   [D, J, K, D, D, D, D, D, D, J, J]   
  2  LIGHTCURVE_Q0    1 BinTableHDU     63   469R x 13C   [J, D, D, D, D, D, D, D, J, D, D, D, D]   
  3  APERTURE_Q0    1 ImageHDU        10   (17, 94)   uint8   
  4  LIGHTCURVE_Q1    1 BinTableHDU     63   1624R x 13C   [J, D, D, D, D, D, D, D, J, D, D, D, D]   
  5  APERTURE_Q1    1 ImageHDU        10   (14, 139)   uint8   
  6  LIGHTCURVE_Q2    1 BinTableHDU     63   4081R x 13C   [J, D, D, D, D, D, D, D, J, D, D, D, D]   
  7  APERTURE_Q2    1 ImageHDU        10   (14, 184)   uint8   
  8  LIGHTCURVE_Q3    1 BinTableHDU     63   4135R x 13C   [J, D, D, D, D, D, D, D, J, D, D, D, D]   
  9  APERTURE_Q3    1 ImageHDU        10   (13, 135)   uint8   
 10  LIGHTCURVE_Q4    1 

The keyword `"QDETECT"` has an 18-long string (1 per quarter) with `0` if no PSF detection, `1` if `PSFFRAC > 0.5`, and `2` if `PSFFRAC<0.5`. Only quarters with `1` are stitched. This is useful to check when a source has been detected and if that quarter data was used for stitching.

In [21]:
hdul["PRIMARY"].header["QDETECT"]

'111111111111111112'

In this case all quarters have `1` but quarter 17 (last one) which has a `2` meaning partial pixel data was used to fit do PSF photometry and could be low quality. Then the rows that correspond to quarter 17 in the stitched light curve are blank for the PSF photmetry columns

In [47]:
Table(hdul["LIGHTCURVE_STITCHED"].data)[63990:63995]

TIME,CADENCENO,QUARTER,FLUX,FLUX_ERR,SAP_FLUX,SAP_FLUX_ERR,PSF_FLAT_FLUX,PSF_FLAT_FLUX_ERR,SAP_QUALITY,FLATTEN_MASK
float64,int32,int64,float64,float64,float64,float64,float64,float64,int32,int32
1557.9384321370162,70913,16,70573.134480144,3.8212731352363294,65067.29798221754,7.219687527783014,70452.13697580327,4.002641870109293,128,0
1557.9588665426709,70914,16,70544.53096341126,3.819104716042227,65132.944760782804,7.221658421609254,70418.0530253074,4.035495852740489,0,0
1559.2257784404792,70976,17,nan,nan,65297.13222754753,9.097476060936833,nan,nan,0,0
1559.2462127641775,70977,17,nan,nan,65370.01541602101,9.101001107048047,nan,nan,0,0
1559.266647187993,70978,17,nan,nan,65262.62563213866,9.09621512143301,nan,nan,8320,8320


But you can still access Q 17 data in the corresponding HDU extension

In [50]:
hdul["LIGHTCURVE_Q17"].header["PSFFRAC"]

0.004945018715731771

In [49]:
Table(hdul["LIGHTCURVE_Q17"].data)

CADENCENO,TIME,FLUX,FLUX_ERR,SAP_FLUX,SAP_FLUX_ERR,CENTROID_COLUMN,CENTROID_ROW,SAP_QUALITY,SAP_BKG,RED_CHI2,PSF_FLUX_NOVA,PSF_FLUX_ERR_NOVA
int32,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
70976,1559.2257784404792,59992.34836278086,6.307520626279437,83934.5625,9.05512809753418,517.923380587951,488.34535926787044,0,2968.363525390625,3490970.897407331,53097.711650794496,5.875478414871276
...,...,...,...,...,...,...,...,...,...,...,...,...
72530,1590.9810741688125,44394.93316384949,5.405614433799843,96521.9609375,9.484403610229492,517.9425648110227,488.36117368921396,128,2512.904296875,5026911.895235132,59323.522504357556,6.072948452795919
72531,1591.0015089050867,44945.24978767853,5.425995001713836,96311.4609375,9.477392196655273,517.9422934491499,488.3612976682722,0,2513.47607421875,4981247.715385974,59276.31027053693,6.070520899236365


Here is another case:

In [63]:
lc = lk.search_lightcurve("290.578158 +52.001121", mission="Kepler", author="KBONUS-BKG").download()
lc

/Users/jorgemarpa/Work/public_repos/lightkurve/src/lightkurve/search.py:480: LightkurveWarning: Warning: 19 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
PSF Fraction is low, indicating not all of the flux is captured in the PSF model. This data may be unreliable.


time,flux,flux_err,cadenceno,quarter,sap_flux,sap_flux_err,psf_flat_flux,psf_flat_flux_err,sap_quality,flatten_mask
,electron / s,electron / s,,,electron / s,electron / s,electron / s,electron / s,,
Time,float64,float64,int32,int64,float64,float64,float64,float64,int32,int32
120.53883484611288,———,———,568,0,2580.7904952848494,1.829539107554908,———,———,0,0
...,...,...,...,...,...,...,...,...,...,...
130.24507382605225,———,———,1043,0,2576.598060949575,1.836613058506873,———,———,0,0


In [64]:
lc.meta

{'EXTNAME': 'PRIMARY',
 'CHECKSUM': 'BViDBVgABVgABVgA',
 'DATASUM': '0',
 'SIMPLE': True,
 'BITPIX': 8,
 'NAXIS': 0,
 'EXTEND': True,
 'NEXTEND': 2,
 'EXTVER': 1,
 'DOI': '10.17909/7jbr-w430',
 'HLSPID': 'KBONUS-BKG',
 'HLSPLEAD': 'Jorge martinez-Palomera',
 'HLSPVER': 'V1.0',
 'LICENSE': 'CC BY 4.0',
 'LICENURL': 'https://creativecommons.org/licenses/by/4.0/',
 'ORIGIN': 'PSFMACHINE',
 'DATE': '2022-09-05',
 'CREATOR': 'lightkurve.LightCurve.to_fits()',
 'TELESCOP': 'Kepler',
 'INSTRUME': 'Kepler Photometer',
 'FILTER': 'KEPLER',
 'TIMESYS': 'TDB',
 'OBJECT': 'KIC 12785454',
 'KEPLERID': 12785454,
 'RADESYS': 'ICRS',
 'RA_OBJ': 290.578157681737,
 'DEC_OBJ': 52.00112124914597,
 'EQUINOX': 2016.0,
 'PROCVER': '2.0.9',
 'VERSION': '1.1.2',
 'APERTURE': 'PSF + SAP',
 'MISSION': 'Kepler',
 'OBSMODE': 'long cadence',
 'XPOSURE': 1765.464,
 'MJD-BEG': 54953.03907252848,
 'MJD-END': 56423.50139254052,
 'LABEL': 'KIC 12785454',
 'TARGETID': 'Gaia DR3 2139265971112336000',
 'KEPMAG': 14.5609998

Now KIC 12785454 was only detected in Q0 and with a PSF fraction < 10%. Let's check Q0 ligth curve

In [62]:
lcq = lk.search_lightcurve("290.578158 +52.001121", mission="Kepler", author="KBONUS-BKG")[1].download()
lcq

PSF Fraction is low, indicating not all of the flux is captured in the PSF model. This data may be unreliable.


time,flux,flux_err,cadenceno,sap_flux,centroid_col,sap_flux_err,centroid_column,centroid_row,sap_quality,sap_bkg,red_chi2,psf_flux_nova,psf_flux_err_nova
,electron / s,electron / s,,electron / s,pix,electron / s,pix,pix,,,,electron / s,electron / s
Time,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
120.53883484611288,31887.98342857861,12.090962387191295,568,2567.759765625,45.06559045423762,1.8064988851547241,45.06559045423762,527.4526073079362,0,166.02459716796875,0.0356540572843177,31685.645533284674,12.01423186530004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130.24507382605225,31838.726684815876,11.996584636222373,1043,2587.980224609375,45.06051799632405,1.8093091249465942,45.06051799632405,527.4500294121044,0,161.2256622314453,0.03493957607270248,31935.14508137068,12.032919110991761


In [67]:
fits.open(lcq.filename)[2].header["PSFFRAC"]

0.08461206273357356